
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to_TF_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to_TF_Lite.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



# TensorFlow Lite Conversion #

<img src="https://miro.medium.com/max/1400/1*AO4WEkrSRz4Xb7qzkbqcvg.png"  alt="Visit my Blog"> 

<img src="https://lh3.googleusercontent.com/vvBAqSnXyg3h9yS0JLyVehhV-e__3NFbZ6q7Ft-rEZp-9wDTVZ49yjuYJwfa4jQZ-RVnChHMr-DDC0T_fTxVyQg3iBMD-icMQooD6A=w630-rw" alt="Visit my Blog">

Once we have our trained model, there is one final step we must take before we are ready to start testing the model. 

We have to convert the frozen graph into a TF Lite model which our inference application can use. The model is quantized, which means it is compatible with Edge TPU devices such as Google Coral.

#Setup

### Connect to Google Drive

In [10]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Install required libraries
Be sure to click on the **restart** button after running this

In [1]:
!pip install -U --pre tensorflow=="1.*"
!pip install tf_slim
!apt-get update && apt-get upgrade -y
!pip install numpy==1.16 #This version is required due to a bug. After installing this version, the runtime has to be restarted.

     |████████████████████████████████| 110.5MB 47kB/s 
     |████████████████████████████████| 512kB 40.3MB/s 
     |████████████████████████████████| 3.8MB 42.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=11edfebdd13acedab1aacf39044c321d03052ed1e8267f14e3187ba5c0d28abd
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: 

Make sure you have `pycocotools` installed

In [1]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [2]:
!apt-get install python-pil python-lxml 

!pip install Cython
!pip install contextlib2
!pip install jupyter
!pip install matplotlib
!pip install wget

import os
import pathlib
import wget


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

os.environ['PYTHONPATH'] += ":/content/models/research/:/content/models/research/slim"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 17 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic

Compile protobufs and install the object_detection package

In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [4]:
%%bash 
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1113489 sha256=bc416facba18f1667e2bed296e42f58f040659ba8bcd5fa334273565c5e066a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-b6606fts/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


# Imports & Configuration

In [16]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import *
wget.download(PRETRAINED_MODEL_CONFIG,CONFIG_DIR)
wget.download("https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/face_ssd_pipeline.config",CONFIG_DIR)

--2020-06-16 17:49:47--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5534 (5.4K) [text/plain]
Saving to: ‘constants.py.2’

constants.py.2      100%[===================>]   5.40K  --.-KB/s    in 0s      

2020-06-16 17:49:48 (67.1 MB/s) - ‘constants.py.2’ saved [5534/5534]



'/content/tensorflow/workspace/Face-Mask-Detection/training/config//face_ssd_pipeline.config'

## Copy dataset and extract

In [23]:
%cd /
!gsutil cp {DRIVE_DEV}model_checkpointsv1.zip ./
!unzip model_checkpointsv1.zip


/
Copying file:///content/drive/My Drive/Machine Learning/Projects/Face-Mask-Detection/Dev/model_checkpointsv1.zip...
-
Operation completed over 1 objects/265.4 MiB.                                    
Archive:  model_checkpointsv1.zip
   creating: content/training/
  inflating: content/training/model.ckpt-11244.meta  
  inflating: content/training/model.ckpt-7477.data-00000-of-00001  
   creating: content/training/eval_0/
  inflating: content/training/eval_0/events.out.tfevents.1592316220.8ef3e07ceeae  
  inflating: content/training/model.ckpt-12496.index  
  inflating: content/training/model.ckpt-9988.index  
  inflating: content/training/model.ckpt-9988.meta  
  inflating: content/training/model.ckpt-12496.meta  
  inflating: content/training/model.ckpt-11244.data-00000-of-00001  
  inflating: content/training/model.ckpt-9988.data-00000-of-00001  
  inflating: content/training/model.ckpt-7477.index  
  inflating: content/training/model.ckpt-8732.index  
  inflating: content/training

In [26]:
#!gsutil cp {DRIVE_DEV}model_checkpointsv1.zip /content/
!unzip -j model_checkpointsv1.zip -d  {DATASET_DIR}

Archive:  model_checkpointsv1.zip
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/dataset/model.ckpt-11244.meta  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/dataset/model.ckpt-7477.data-00000-of-00001  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/dataset/events.out.tfevents.1592316220.8ef3e07ceeae  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/dataset/model.ckpt-12496.index  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/dataset/model.ckpt-9988.index  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/dataset/model.ckpt-9988.meta  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/dataset/model.ckpt-12496.meta  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/dataset/model.ckpt-11244.data-00000-of-00001  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/dataset/model.ckpt-9988.data-00000-of-00001  
  inflating: /content/tensorflow/workspace/Fac

# Conversions

## Export inference graph

Use this option if you want to convert to TFjs later

In [27]:
!python /content/models/research/object_detection/export_inference_graph.py \
  --pipeline_config_path={CONFIG_DIR}face_ssd_pipeline.config \
  --trained_checkpoint_prefix=/content/training/model.ckpt-12496 \
  --output_directory=/content/training/graphs/inf \
  --add_postprocessing_op=true \
  --input_type=image_tensor \
  

Instructions for updating:
Please use `layer.__call__` method instead.
W0616 18:18:27.743030 140079729260416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0616 18:18:30.225367 140079729260416 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0616 18:18:30.267732 140079729260416 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0616 18:18:30.306137 140079729260416 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

## Export TF Lite SSD inference graph

Use this option for TF Lite conversion

In [ ]:
!python {BASE_DIR}/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path="{CONFIG_DIR}/pipeline.config" \
--trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-19611" \
--output_directory="{TF_LITE_GRAPH}" \
--add_postprocessing_op=true

2020-06-11 21:21:50.333863: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-06-11 21:21:54.593521: I tensorflow/tools/graph_transforms/transform_graph.cc:317] Applying strip_unused_nodes


## Convert graph to TF Lite

In [ ]:
!tflite_convert \
--output_file="{TF_LITE_MODEL}/detect2.tflite" \
--graph_def_file="{TF_LITE_GRAPH}/tflite_graph.pb" \
--inference_type=QUANTIZED_UINT8 \
--input_arrays="{INPUT_TENSORS}" \
--output_arrays="{OUTPUT_TENSORS}"  \
--mean_values=128 \
--std_dev_values=128 \
--input_shapes=1,300,300,3 \
--change_concat_input_ranges=false \
--allow_nudging_weights_to_use_fast_gemm_kernel=true \
--allow_custom_ops \
--output_format=TFLITE \
--default_ranges_min=0 \
--default_ranges_max=255

2020-06-11 21:22:11.079323: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


# Save 

## Archive the inference graph

In [ ]:
!zip -r inf-graphv1.zip {INF_GRAPH}
!gsutil cp inf-graphv1.zip {DRIVE_DEV}

/content/training_demo
--2020-06-05 08:59:06--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/ssd_mobilenet_v2_coco.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4766 (4.7K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_coco.config’

ssd_mobilenet_v2_co 100%[===================>]   4.65K  --.-KB/s    in 0s      

2020-06-05 08:59:06 (46.5 MB/s) - ‘ssd_mobilenet_v2_coco.config’ saved [4766/4766]



## Archive the TF Lite graph

In [ ]:
!zip -r tflite-graphv1.zip {TF_LITE_GRAPH}
!gsutil cp tflite-graphv1.zip {DRIVE_DEV}

  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/tf_lite_graph/ (stored 0%)
  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/tf_lite_graph/tflite_graph.pb (deflated 8%)
  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/tf_lite_graph/tflite_graph.pbtxt (deflated 56%)
Copying file://tflite-graphv1.zip...
-
Operation completed over 1 objects/39.5 MiB.                                     


## Archive the TF Lite model

In [ ]:
!zip -r tflite-modelv1.zip {TF_LITE_MODEL}
!gsutil cp tflite-modelv1.zip {DRIVE_DEV}

  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained-models/tflite/ (stored 0%)
  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained-models/tflite/detect2.tflite (deflated 39%)
Copying file://tflite-modelv4.zip...
/ [1 files][  2.8 MiB/  2.8 MiB]                                                
Operation completed over 1 objects/2.8 MiB.                                      
